In [5]:
import requests
import re
import nltk
from nltk import tokenize
nltk.download('punkt') # Download before running the code.  

S = requests.Session()
URL = "https://nl.wikipedia.org/w/api.php"

f = open("firstsections.txt", "w")
g = open("sws.txt", "w")

# Gives a list of all articles inside a Wikipedia category. 
def getCategoryList(category):
  listArticlesCategory = []
  PARAMS = {
      'action': "query",
      'list': "categorymembers",
      'cmtitle': "Category:" + category,
      'cmlimit': 440, # The limit of categories.
      'cmprop':'title|sortkey',
      # 'cmstarthexsortkey': '', #If you fill in the sortkey, the code will begin running at that sortkey. 
      'format': "json"
  }

  R = S.get(url=URL, params=PARAMS)
  data = R.json()

  for element in data['query']['categorymembers']:
    # We do not want Wikipedia categories, only the articles inside a category.
    if not element['title'].startswith('Categorie:'): 
      listArticlesCategory.append(element['title'])
  
  return listArticlesCategory

# Gives a list of the parent categories from a specific Wikipedia article. 
def getParentCat(subject):
  parentCat = []
  PARAMS = {
      "action":"query",
      "format":"json", 
      "titles":subject,
      "prop":"categories",
  }
  R = S.get(url=URL, params=PARAMS)
  data = R.json()

  for value in data["query"]["pages"].values():
    if 'categories' in value: # Not all Wikipedia articles have a parent category.
      for dictionary in value['categories']:
        if not dictionary['title'].startswith('Categorie:Wikipedia:'): 
          # Wikipedia also knows hidden categories, we do not want those.
          cat = dictionary['title']
          parentCat.append(cat)         
      return parentCat
    else:
      return subject + " does not have any parent categories."

# Given a certain Wikipedia article, will return a list of 
# sentences containing the title of that Wikipedia article.
def getSentencesWithSubject(subject):
  allSections = ""
  numberSections = ['0']

  PARAMS = {
      'action': 'parse',
      'page': subject,
      'prop': 'sections',
      'format': 'json'
  }
  R = S.get(url=URL, params=PARAMS)
  data = R.json()

  # Retrieves all sections.
  if not data['parse']['sections'] == []:
    for sectionnumber in data['parse']['sections']:
      if sectionnumber['index'] != '':
        numberSections.append(sectionnumber['index'])
        for indexnumber in numberSections:
          PARAMS = {
          'action': "parse",
          'prop': "wikitext",
          'section': indexnumber,
          'page': subject,
          'format': "json"
          }

          R = S.get(url=URL, params=PARAMS)
          data = R.json()

          nextSection = data['parse']['wikitext']['*']
          nextSection = cleanSection(nextSection, subject)
           
          allSections += nextSection + " "
  else:
    PARAMS = {
    'action': "parse",
    'prop': "wikitext",
    'section': 0,
    'page': subject,
    'format': "json"
    }

    R = S.get(url=URL, params=PARAMS)
    data = R.json()
    allSections = data['parse']['wikitext']['*']
    allSections = cleanSection(allSections, subject)
      
  # Makes a list of all sentences containing the subject,
  # where links take the form of [[c]] or [[a|b]]. 
  # An example of [[a|b]] is [[China|Chinese]], in the text it is displayed 'Chinese',
  # but the link will go to the Wikipedia article 'China'. 
  sentencesWithSubjectLine = []
  for sentence in tokenize.sent_tokenize(allSections):
    if "[[" + subject + "]]" in sentence:
      sentencesWithSubjectLine.append(sentence)  
 
  # Finds all NEs (that either look like [[c]] or [[a|b]])
  # in the listsentencesWithSubjectLine. 
  # Including NEs such as "'s-Hertogenbosch", "'s Gravenshage" and "'t Vossenhol".
  # The Ł-character is added, because of the occurence of a Polish city in one of the articles.
  NEsInSentencesLine = []
  for sentence in sentencesWithSubjectLine:
    NEsInSentencesLine.append(re.findall(r"\[\[(?:'[a-z][\s-])?[A-ZŁ].*?\]\]", sentence))
     
  # Changes all [[a|b]] in [[b]].
  for listLinks in NEsInSentencesLine:
    for link in listLinks:
      if link != '':
        if re.match("(.*?)\|(.*?)", link):
          linkAfter = re.sub("\[\[(.*?)\|", "[[", link)
          allSections = re.sub(re.escape(link), linkAfter, allSections)
  
  # Makes a list of all sentences containing the subject,
  # where links take the form of [[a]]. 
  sentencesWithSubject = []
  for sentence in tokenize.sent_tokenize(allSections):
    if "[[" + subject + "]]" in sentence:
      sentencesWithSubject.append(sentence)
      
  # Creates the dictionary {named entity: label}.
  NEplusTags = {}
  for listLinks in NEsInSentencesLine:
    for link in listLinks:
      if link != '':
        if re.match("(.*?)\|(.*?)", link):
          link = re.sub("[\[\[\]\]]", "", link)
          linkAfter = re.sub("^((.*?)\|)", "", link)
          linkBefore = re.sub("\|(.*?)$", "", link)
          if getURL(linkBefore) != " does not have a link.":
            NEplusTags[linkAfter] = getNERtag(linkBefore)   
        else:
          link = re.sub("[\[\[\]\]]", "", link)
          if getURL(link) != " does not have a link.":
            NEplusTags[link] = getNERtag(link)
  
  return sentencesWithSubject, NEplusTags

def cleanSection(section, subject):
  # Deletes the infobox, categories, files, links, subheaders, tables, footnotes and images, etc. 
  changes = [
    ('\{\| class="wikitable"\n(\|.*\n)*\|\}', ''), 
    ('\[\[Categorie:(.*?)\]\]\n?', ''),
    ('\[\[Bestand(.*?)\]\](?:\s-)?\n', ''),
    ('\[\[Bestand(.*?)\]\]$', ''),
    ('\[\[Bestand(.*?)\]\](\n)?', ''),
    ('\[\[Image:(.*?)\]\]\n', ''),
    ('\[\[Image:(.*?)\]\]', ''),
    ('\[\[Afbeelding(.*?)\]\]\n', ''),
    ('\[\[File(.*?)\]\]\n', ''),
    ('<ref>\[.*?\n?.*?\].*?<\/ref>', ''),
    ('<ref name="\w*"/>', ''),    
    ('<\/?(.*?)>(.*?)<\/?(.*?)>', ''),
    ('\{\{.*?\}\}', ''),
    ('\{\{(.*?)\n\|.*\n(?:\s*\n)?(\|.*\n)*\|?\}\}\n*', ''),
    ('==(.*?)==\s', ''), 
    ('\[http.*?\]\n?', ''),
    ('\*\s?\[http.*?\]', ''), 
    ("\'\'\'", ""),
    ("\'\'", ""),
    ('\n',' ')
  ]
  
  for old, new in changes:
    section = re.sub(old, new, section)

  # Finds all [[a|b]] where b starts with a lowercase letter or integer. 
  # Deletes the a| in [[a|b]] links. 
  for link in re.findall(r"\[\[([^]]+)\|'?\.?[a-z0-9].*?\]\]", section):
    section = re.sub(re.escape(link) + r"\|", "", section)
  
  # Finds all links that start with a lowercase letter or integer, 
  # and deletes the [[]] around those links. 
  for link in re.findall(r"\[\['?\.?[a-z0-9].*?\]\]", section):
    link = re.sub("[\[\[\]\]]", "", link)
    section = re.sub(r"\[\[" + re.escape(link) + r"\]\]", re.escape(link), section)
    
  # Words after a bullet * always start with a capital letter,
  # even though they do not refer to a named entity. 
  # Code below will delete the [[]] that follow after a *. 
  # This code needs improvement, 
  # because it will also delete the [[]] of a named entity after a *.
  for link in re.findall(r"\*\[\[(.*?)\]\]\.?", section):
    link = re.sub("[\[\[\]\]]", "", link)
    section = re.sub(r"\[\[" + re.escape(link) + r"\]\]", re.escape(link), section)
    
  # Sometimes the text contains words like [[Nederland]]se or [[Bloedbank]]en. 
  # The link will go to the article that is inside the [[]], 
  # but the NE is in that case not complete.  
  # Code below will change [[a]]b to [[ab]].
  for link in re.findall(r"\[\[(\b[A-Z].*?)\]\]", section):
    for incomplNE in re.findall(r"\[\[" + re.escape(link) + r"\]\][a-z]+", section):
      complNE = re.sub("[\[\[\]\]]", "", incomplNE)
      section = re.sub(re.escape(incomplNE), "[[" + complNE + "]]", section)
  
  # If the word is being linked to a non-existing page, the [[]] of the link are deleted.
  for link in re.findall(r"\[\[\s?(?:'[a-z][\s-])?[A-ZŁ].*?\]\]", section):
    if re.match("(.*?)\|(.*?)", link):
      linkBefore = re.sub("\|(.*?)$", "", link)
      linkClean = re.sub("[\[\[\]\]]", "", linkBefore)
      if getURL(linkClean) == " does not have a link.":
        section = re.sub(re.escape(linkBefore), linkClean, section)
    else:
      linkClean = re.sub("[\[\[\]\]]", "", link)
      if getURL(linkClean) == " does not have a link.":
        section = re.sub(re.escape(link), linkClean, section)
  
  # The title of the article is also a NE, so it is put between [[]].
  section = re.sub(" " + re.escape(subject) + " ", " [[" + subject + "]] ", section)
      
  return section

# Will retrieve the first section of a Wikipedia article.
def getFirstSection(subject):
  linksInSectionURL = []
  PARAMS = {
  'action': "parse",
  'prop': "wikitext",
  'section': 0,
  'page': subject,
  'format': "json"
  }

  R = S.get(url=URL, params=PARAMS)
  data = R.json()
  section = data['parse']['wikitext']['*']

  section = cleanSection(section, subject)
    
  # All NEs are put into the list linksInSection.
  linksInSection = re.findall(r"\[\[(?:'[a-z][\s-])?[A-ZŁ].*?\]\]", section)
  
  # All NEs are stripped from [[]] and added to the list linksInSectionURL.  
  for link in linksInSection:
    link = re.sub(r"\[\[|\]\]", "", link)
    linksInSectionURL.append(link)
  
  # If NEs in linksInSectionURL take the form of [[a|b]], a| will be deleted, 
  # so their form will be [[b]].
  for link in linksInSectionURL:
    if re.match("(.*?)\|(.*?)", link):
      linkAfter = re.sub("^((.*?)\|)", "", link)
      section = re.sub(re.escape(link), linkAfter, section)
  
  # The dictionary {named entity: label} is created.
  NEplusTags = {}
  for link in linksInSectionURL:
    if re.match("(.*?)\|(.*?)", link):
      linkAfter = re.sub("^((.*?)\|)", "", link)
      linkBefore = re.sub("\|(.*?)$", "", link)
      if getURL(linkBefore) != " does not have a link.":
        tag1 = getNERtag(linkBefore) 
        NEplusTags[linkAfter]= tag1
    elif getURL(link) != " does not have a link.":
      tag2 = getNERtag(link)
      NEplusTags[link] = tag2  

  return (section, NEplusTags)

# Determines the category of a named entity.
queue = []
def getNERtag(subject):
  result = ''
  queue.append(subject)
  
  while queue and result == '':
    cat = queue.pop(0)
    result = findCat(cat)

  return result
      
# Finds the parent categories of a certain Wikipedia article. 
def findCat(subject): 
  PARAMS = {
      'action': 'query',
      'format': 'json',
      'pageids': getURL(subject),
  }
  
  R = S.get(url=URL, params=PARAMS)
  data = R.json()
  title = data['query']['pages'][getURL(subject)]['title']
  
  parentCategories = getParentCat(title)
  
  if parentCategories == subject + " does not have any parent categories.":
    return 'misc'
  
  sti = ['stichting']
  org = ['organisatie', 'vereniging', 'kerkgenootschap', 'fonds', 'ministerie', 
        'partij', 'omroep', 'krijgsmacht', 'universiteit', 'bestuursorgaan', 'raad', 'instituut',
        'museum', 'instantie', 'bedrijf']
  per = ['persoon', 'hoogleraar', 'wetenschapper', 'schrijver', 'politicus',
        'ontwerper', 'god ']
  loc = ['geografie', 'plaats', 'rijk', 'continent', 'provincie', 
        'meer', 'land in', 'monument', 'bouwwerk', 'planeet']
  eve = ['oorlog', 'themadag', 'evenement', 'natuurramp']
  pro = ['taal', 'dialect', 'prijs', 'boek ', 'schrift', ' dier', 'object', 'lied', 
         'voertuig', 'vliegtuig', 'krant']
  misc = ['stroming', 'volk', 'formaat', 'chemische stof', 'wet ', 'koninkrijk',
         'website', 'onderwijsvorm']
  
  for cat in parentCategories:
    for foundation in sti:
      if foundation in cat.lower():
        queue.clear()
        return 'sti'
  
  for cat in parentCategories:
    for organisation in org:
      if organisation in cat.lower():
        queue.clear()
        return 'org'
    for person in per:
      if person in cat.lower():
        queue.clear()
        return 'per'
    for location in loc:
      if location in cat.lower():
        queue.clear()
        return 'loc'
    for event in eve:
      if event in cat.lower():
        queue.clear()
        return 'eve'
    for product in pro:
      if product in cat.lower():
        queue.clear()
        return 'pro'
    for miscellaneous in misc:
      if miscellaneous in cat.lower():
        queue.clear()
        return 'misc'

  queue.extend(parentCategories)
  
  return ''

# Will get the URL of a certain Wikipedia article.
def getURL(subject):
  PARAMS = {
      'action': "query",
      'titles': subject,
      'format': "json",
      'redirects': True
  }
  R = S.get(url=URL, params=PARAMS)
  data = R.json()
  
  for pageid in data['query']['pages'].keys():
    if pageid == "-1":
      return " does not have a link."
    else:
      return pageid

def cleanWpL(text, dictOfNEsAndTags):  
  changes = [
    (r"([?!:/;\"]+)", r" \1"), 
    (r"([()’‘\*]+)", r" \1 "),
    (r"([.,]\s)", r" \1 "),
    (r"\.$", r" . "),
    (r"\\", ""),
    (r"\]\]'", "]] '"), # for 't Vossenhol
    ("  ", " ")
  ]
  
  for old, new in changes:
    text = re.sub(old, new, text) 
  
  # Deals with NEs that look like [[a]]-[[b]], [[a]]-b, a-[[b]].
  NEsHyphen = re.findall(r"\[\[.*?\]\]-(?:\[\[)*\w*(?:\]\])*", text)
  for ne in NEsHyphen:
    neNew = re.sub(r"-", " - ", ne)
    text = re.sub(re.escape(ne), neNew, text)
   
  # Deals with abbreviations.
  abbreviations = re.findall(r"[A-Z]\s\.(?:[A-Z]\s\.)*\s\w*", text)
  for abrv in abbreviations:
    abrvNew = re.sub(r"\s\.", ".", abrv)
    text = re.sub(re.escape(abrv), abrvNew, text)
    
  # Deals with various non-alphanumeric characters inside of NEs.
  allNEs = re.findall(r"\[\[(?:.*?)\]\]", text)
  for ne in allNEs:
    if re.search("\(.*?\)", ne):
      neNew = re.sub("\s\)\s", ")", ne)
      neNewer = re.sub("\(\s", "(", neNew)
      text = re.sub(re.escape(ne), neNewer, text)
    if ' - ' in ne:
      neNew = re.sub("[\[\[\]\]]", "", ne)
      if neNew not in dictOfNEsAndTags:
        neNew = re.sub('\s-\s', '-', ne)
        text = re.sub(re.escape(ne), neNew, text)
    if re.search("\[\[\[\[.*?\]\]", ne):
      neNew = re.sub("[\[\[\]\]]", "", ne)
      text = re.sub(re.escape(ne), neNew, text)
    if re.search(re.escape(ne) + "\]\]", text):
      text = re.sub("[\]\]]", "", text)
    for n in re.findall(r"\s[?!:,\".’/;]+", ne):
      neNew = re.sub(r"\s", "", n)
      neNewer = re.sub(re.escape(n), neNew, ne)
      text = re.sub(re.escape(ne), neNew, text)
      
  return text

# Will turn a text into a token-per-line format.

'''
Because of the use of a dictionary {named entity: label}, 
named entities that have different labels in different contexts, 
such as 'Bart Smit' either being a PER or an ORG, 
will only have one key and one value. 
So, if the first occurence of 'Bart Smit' is annotated as ORG, 
all the other occurences in the text will also be annotated as ORG.
Instead of a dictionary, an array would be a better option.
'''
def wordPerLine(text, dictOfNEsAndTags):
  #print(text)
  #print(dictOfNEsAndTags)
  
  # Cleans the text so that it is ready to be tokenized. 
  text = cleanWpL(text, dictOfNEsAndTags)
  
  # Splits the text in sentences.
  listOfSentences = re.split(r'(?<= \.) ', text)

  output = []
  search = ""
  inTag = False
  
  # Will add the BIO-encoding.
  for sentence in listOfSentences:
    for w in sentence.split(" "):
      outTag = False
      rest = w

      if rest[:2] == "[[":
        rest = rest[2:]
        inTag = True
      if rest[-2:] == "]]":
        rest = rest[:-2]
        outTag = True

      if inTag:
        search += rest
        if outTag:
          val = dictOfNEsAndTags[search]
          for word in search.split()[:1]:
            output.append(word + "\tB-" + val)
          for word in search.split()[1:]:
            output.append(word + "\tI-" + val)
          inTag = False
          search = ""
        else:
          search += " "
      
      elif rest != "":
        output.append(rest + "\tO")
    
    if output:
      if output[-1].endswith('.\tO'):
        output.extend("\n")
 
  return output

  
'''
for foundation in getCategoryList("Nederlandse stichting"):
  for sentence in getSentencesWithSubject(foundation)[0]:
    for w in wordPerLine(sentence, getSentencesWithSubject(foundation)[1]):
      g.write(w)
      g.write("\n")
    g.write("\n")
   
g = open('sws.txt')
content = g.read()
g.close()
newContent = content.replace('\n\n\n', '\n')
g = open('sws.txt', 'w')
g.write(newContent)


for foundation in getCategoryList("Nederlandse stichting"):
  for w in wordPerLine(getFirstSection(foundation)[0], getFirstSection(foundation)[1]):
    for x in w:
      f.write(x)
    f.write("\n")

f = open('firstsections.txt')
content = f.read()
f.close()
newContent = content.replace('\n\n', '\n')
f = open('firstsections.txt', 'w')
f.write(newContent)

'''
g.close()
f.close()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
